In [1]:
%%capture
!pip install datasets==1.14
!pip install transformers==4.11.3
!pip install librosa

In [2]:
model_checkpoint = "facebook/wav2vec2-base"
batch_size = 16

In [3]:
!pip install pyarrow==13.0.0
!pip install datasets==2.14.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 48.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 13.0.0 which is incompatible.
dask-expr 1.1.21 requires pyarrow>=14.0.1, but you have pyarrow 13.0.0 which is incompatible.
pandas-gbq 0.23.1 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━

In [4]:
from datasets import load_dataset, load_metric

In [5]:
dataset = load_dataset("AbstractTTS/CREMA-D")
metric = load_metric("accuracy")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/7442 [00:00<?, ? examples/s]

<ipython-input-5-2a382cdf5f07>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'angry', 'sad', 'disgust', 'fear', 'neutral', 'happy', 'gender', 'transcription', 'major_emotion', 'speaking_rate', 'pitch_mean', 'pitch_std', 'rms', 'relative_db'],
        num_rows: 7442
    })
})

In [7]:
dataset['train']

Dataset({
    features: ['file', 'audio', 'angry', 'sad', 'disgust', 'fear', 'neutral', 'happy', 'gender', 'transcription', 'major_emotion', 'speaking_rate', 'pitch_mean', 'pitch_std', 'rms', 'relative_db'],
    num_rows: 7442
})

In [8]:
from sklearn.model_selection import train_test_split
import numpy as np

train_indices, temp_indices = train_test_split(np.arange(dataset['train'].num_rows), test_size=0.3, random_state=42)

train = dataset['train'].select(train_indices)
temp = dataset['train'].select(temp_indices)

val_indices, test_indices = train_test_split(np.arange(temp.num_rows), test_size=0.5, random_state=42)
val = temp.select(val_indices)
test = temp.select(test_indices)

from datasets import DatasetDict

dataset_splitted = DatasetDict({
    'train': train,
    'validation': val,
    'test': test
})

In [9]:
dataset_splitted

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'angry', 'sad', 'disgust', 'fear', 'neutral', 'happy', 'gender', 'transcription', 'major_emotion', 'speaking_rate', 'pitch_mean', 'pitch_std', 'rms', 'relative_db'],
        num_rows: 5209
    })
    validation: Dataset({
        features: ['file', 'audio', 'angry', 'sad', 'disgust', 'fear', 'neutral', 'happy', 'gender', 'transcription', 'major_emotion', 'speaking_rate', 'pitch_mean', 'pitch_std', 'rms', 'relative_db'],
        num_rows: 1116
    })
    test: Dataset({
        features: ['file', 'audio', 'angry', 'sad', 'disgust', 'fear', 'neutral', 'happy', 'gender', 'transcription', 'major_emotion', 'speaking_rate', 'pitch_mean', 'pitch_std', 'rms', 'relative_db'],
        num_rows: 1117
    })
})

In [10]:
dataset = dataset_splitted

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'angry', 'sad', 'disgust', 'fear', 'neutral', 'happy', 'gender', 'transcription', 'major_emotion', 'speaking_rate', 'pitch_mean', 'pitch_std', 'rms', 'relative_db'],
        num_rows: 5209
    })
    validation: Dataset({
        features: ['file', 'audio', 'angry', 'sad', 'disgust', 'fear', 'neutral', 'happy', 'gender', 'transcription', 'major_emotion', 'speaking_rate', 'pitch_mean', 'pitch_std', 'rms', 'relative_db'],
        num_rows: 1116
    })
    test: Dataset({
        features: ['file', 'audio', 'angry', 'sad', 'disgust', 'fear', 'neutral', 'happy', 'gender', 'transcription', 'major_emotion', 'speaking_rate', 'pitch_mean', 'pitch_std', 'rms', 'relative_db'],
        num_rows: 1117
    })
})

In [12]:


for split in dataset:
    dataset[split] = dataset[split].rename_column("major_emotion", "label")


In [13]:

for split in dataset:
  columns_to_keep = ['audio', 'label']
  columns_to_remove = [col for col in dataset[split].column_names if col not in columns_to_keep]
  dataset[split] = dataset[split].remove_columns(columns_to_remove)



In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'label'],
        num_rows: 5209
    })
    validation: Dataset({
        features: ['audio', 'label'],
        num_rows: 1116
    })
    test: Dataset({
        features: ['audio', 'label'],
        num_rows: 1117
    })
})

In [15]:
dataset["test"][1000]

{'audio': {'path': '1028_ITS_NEU_XX.wav',
  'array': array([-0.00689697, -0.00628662, -0.00643921, ...,  0.        ,
          0.        ,  0.        ]),
  'sampling_rate': 16000},
 'label': 'neutral'}

In [16]:
dataset["train"]['label'][0]

'anger'

In [18]:
id2label = {0:'anger',1:'sad',2:'disgust',3:'fear',4:'neutral',5:'happy'}
id2label[5]

'happy'

In [19]:
label2id = {value: key for key, value in id2label.items()}
label2id["anger"]

0

In [20]:
def map_labels(example):
    if example['label'] not in label2id:
        raise ValueError(f"Label '{example['label']}' not found in label2id dictionary.")
    example['label'] = label2id[example['label']]
    return example

dataset = dataset.map(map_labels)


Map:   0%|          | 0/5209 [00:00<?, ? examples/s]

Map:   0%|          | 0/1116 [00:00<?, ? examples/s]

Map:   0%|          | 0/1117 [00:00<?, ? examples/s]

In [21]:
dataset["train"]['label'][0]

0

In [22]:
dataset["test"][1000]["audio"]

{'path': '1028_ITS_NEU_XX.wav',
 'array': array([-0.00689697, -0.00628662, -0.00643921, ...,  0.        ,
         0.        ,  0.        ]),
 'sampling_rate': 16000}

In [23]:
import random
from IPython.display import Audio, display

for _ in range(5):
    rand_idx = random.randint(0, len(dataset["train"])-1)
    example = dataset["train"][rand_idx]
    audio = example["audio"]

    print(f'Label: {id2label[example["label"]]}')
    print(f'Shape: {audio["array"].shape}, sampling rate: {audio["sampling_rate"]}')
    display(Audio(audio["array"], rate=audio["sampling_rate"]))
    print()

Label: anger
Shape: (42709,), sampling rate: 16000



Label: neutral
Shape: (39506,), sampling rate: 16000



Label: disgust
Shape: (58191,), sampling rate: 16000



Label: anger
Shape: (45912,), sampling rate: 16000



Label: neutral
Shape: (45379,), sampling rate: 16000


In [24]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
feature_extractor

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [25]:
max_duration = 4.0  

In [26]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
    )
    return inputs

In [27]:
preprocess_function(dataset['train'][:5])

{'input_values': [array([0.03663272, 0.04369576, 0.03700446, ..., 0.00020226, 0.00020226,
       0.00020226], dtype=float32), array([-4.8985949e-04,  1.0792131e-03,  1.5985403e-02, ...,
       -9.7591328e-05, -9.7591328e-05, -9.7591328e-05], dtype=float32), array([-5.6405004e-02, -4.9828287e-02, -5.5308886e-02, ...,
        4.5129822e-05,  4.5129822e-05,  4.5129822e-05], dtype=float32), array([ 0.02906499,  0.03273601,  0.02854056, ..., -0.00047797,
       -0.00047797, -0.00047797], dtype=float32), array([0.07463343, 0.13605537, 0.13912646, ..., 0.01935369, 0.02242479,
       0.02242479], dtype=float32)]}

In [28]:
encoded_dataset = dataset.map(preprocess_function, remove_columns=["audio"], batched=True)
encoded_dataset

Map:   0%|          | 0/5209 [00:00<?, ? examples/s]

Map:   0%|          | 0/1116 [00:00<?, ? examples/s]

Map:   0%|          | 0/1117 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values'],
        num_rows: 5209
    })
    validation: Dataset({
        features: ['label', 'input_values'],
        num_rows: 1116
    })
    test: Dataset({
        features: ['label', 'input_values'],
        num_rows: 1117
    })
})

In [29]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-wav2vev2",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    report_to="none"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [46]:
import numpy as np

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions."""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    references = eval_pred.label_ids
    accuracy = metric.compute(predictions=predictions, references=references)["accuracy"]
    return {"accuracy": accuracy}

In [47]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

In [48]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss,Accuracy
0,0.509400,0.900303,0.711470
1,0.503800,0.864253,0.732975
2,0.366600,0.945992,0.707885
4,0.320900,0.861076,0.743728


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-pac

TrainOutput(global_step=200, training_loss=0.3937292838096619, metrics={'train_runtime': 1510.2133, 'train_samples_per_second': 17.246, 'train_steps_per_second': 0.132, 'total_flos': 8.670539861603914e+17, 'train_loss': 0.3937292838096619, 'epoch': 4.9079754601226995})

In [49]:
trainer.evaluate()

{'eval_loss': 0.8610759377479553,
 'eval_accuracy': 0.7437275985663082,
 'eval_runtime': 28.0473,
 'eval_samples_per_second': 39.79,
 'eval_steps_per_second': 1.248,
 'epoch': 4.9079754601226995}